In [ ]:
!unzip train_test_dataset.zip -d data

In [33]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.utils as utils
from google.colab.patches import cv2_imshow


In [6]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
SEED = 42
COLOR_MODE = "grayscale"

train_path = "/content/data/train_test_dataset/train"
test_path =  "/content/data/train_test_dataset/test"

In [16]:
train_data = utils.image_dataset_from_directory(train_path,subset='training' ,validation_split=0.2, seed = 42, image_size=IMG_SIZE, batch_size=BATCH_SIZE,
                                                color_mode=COLOR_MODE)
print('-------------------------')
test_data = utils.image_dataset_from_directory(test_path, seed = 42, image_size=IMG_SIZE, batch_size=BATCH_SIZE, color_mode=COLOR_MODE)


Found 200 files belonging to 5 classes.
Using 160 files for training.
-------------------------
Found 40 files belonging to 5 classes.


In [40]:
class_names = train_data.class_names
class_names

['personA', 'personB', 'personC', 'personD', 'personE']

In [25]:
batch = train_data.as_numpy_iterator().next()

In [ ]:
for i in range(len(batch[0])):
  cv2_imshow(batch[0][i])
  print(train_data.class_names[batch[1][i]])

In [119]:
num_classes = len(class_names)

model = keras.Sequential([
  keras.layers.Rescaling(1./255, input_shape=(*IMG_SIZE, 1)),
  keras.layers.Conv2D(8, (3,3), padding='same', activation='relu', name="CONV1"),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(16, (3,3), padding='same', activation='relu', name="CONV2"),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(32, (3,3), padding='same', activation='relu', name="CONV3"),
  keras.layers.MaxPooling2D(),
  keras.layers.Dropout(0.2),
  keras.layers.Flatten(),
  keras.layers.Dense(32, activation='relu'),
  keras.layers.Dense(num_classes)
])


In [120]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
model.summary()

In [121]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    "checkpoints/e{epoch:02d}-acc{accuracy:.2f}.hdf5",
    monitor='accuracy',
    mode='max',
    save_weights_only=True,
    save_best_only=True)


In [122]:
epochs=7
model.fit(train_data, epochs=epochs, callbacks=[model_checkpoint_callback])

Epoch 1/7
5/5 [==============================] - 6s 963ms/step - loss: 2.0177 - accuracy: 0.2000
Epoch 2/7
5/5 [==============================] - 5s 956ms/step - loss: 1.4090 - accuracy: 0.5938
Epoch 3/7
5/5 [==============================] - 7s 1s/step - loss: 1.1309 - accuracy: 0.6812
Epoch 4/7
5/5 [==============================] - 5s 968ms/step - loss: 0.7897 - accuracy: 0.7063
Epoch 5/7
5/5 [==============================] - 5s 960ms/step - loss: 0.4128 - accuracy: 0.8813
Epoch 6/7
5/5 [==============================] - 5s 964ms/step - loss: 0.1521 - accuracy: 0.9875
Epoch 7/7
5/5 [==============================] - 5s 958ms/step - loss: 0.0369 - accuracy: 1.0000


In [123]:
model.evaluate(test_data)

2/2 [==============================] - 1s 84ms/step - loss: 0.1352 - accuracy: 0.9750


[0.13517233729362488, 0.9750000238418579]

In [131]:
def create_model():
  model = keras.Sequential([
    keras.layers.Rescaling(1./255, input_shape=(*IMG_SIZE, 1)),
    keras.layers.Conv2D(8, (3,3), padding='same', activation='relu', name="CONV1"),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(16, (3,3), padding='same', activation='relu', name="CONV2"),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(32, (3,3), padding='same', activation='relu', name="CONV3"),
    keras.layers.MaxPooling2D(),
    keras.layers.Dropout(0.2),
    keras.layers.Flatten(),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(5)
  ])
  model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

  return model


In [132]:
model2 = create_model()

In [133]:
model2.load_weights('/content/checkpoints/e07-acc1.00.hdf5')

In [135]:
loss, acc = model2.evaluate(test_data, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

2/2 - 1s - loss: 0.1352 - accuracy: 0.9750 - 737ms/epoch - 368ms/step
Untrained model, accuracy: 97.50%
